# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6836a48df0>
├── 'a' --> tensor([[-0.3517,  1.2410,  0.3271],
│                   [ 1.8247, -0.2131, -0.2898]])
└── 'x' --> <FastTreeValue 0x7f6836a48d30>
    └── 'c' --> tensor([[-0.4572, -1.7825, -2.1356, -0.1295],
                        [-0.2411,  0.5536,  0.3949, -0.7374],
                        [-0.1403, -0.3730,  0.8133, -1.4093]])

In [4]:
t.a

tensor([[-0.3517,  1.2410,  0.3271],
        [ 1.8247, -0.2131, -0.2898]])

In [5]:
%timeit t.a

61.1 ns ± 0.0717 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6836a48df0>
├── 'a' --> tensor([[-2.6814, -0.0508,  0.3781],
│                   [-0.8164, -1.9762, -1.5554]])
└── 'x' --> <FastTreeValue 0x7f6836a48d30>
    └── 'c' --> tensor([[-0.4572, -1.7825, -2.1356, -0.1295],
                        [-0.2411,  0.5536,  0.3949, -0.7374],
                        [-0.1403, -0.3730,  0.8133, -1.4093]])

In [7]:
%timeit t.a = new_value

63.6 ns ± 0.027 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3517,  1.2410,  0.3271],
               [ 1.8247, -0.2131, -0.2898]]),
    x: Batch(
           c: tensor([[-0.4572, -1.7825, -2.1356, -0.1295],
                      [-0.2411,  0.5536,  0.3949, -0.7374],
                      [-0.1403, -0.3730,  0.8133, -1.4093]]),
       ),
)

In [10]:
b.a

tensor([[-0.3517,  1.2410,  0.3271],
        [ 1.8247, -0.2131, -0.2898]])

In [11]:
%timeit b.a

55.5 ns ± 0.0236 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.6079, -0.6181, -0.5990],
               [-0.3148, -0.4112, -0.1404]]),
    x: Batch(
           c: tensor([[-0.4572, -1.7825, -2.1356, -0.1295],
                      [-0.2411,  0.5536,  0.3949, -0.7374],
                      [-0.1403, -0.3730,  0.8133, -1.4093]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.0525 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

828 ns ± 0.0602 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.7 µs ± 64.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 126 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 292 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f680beb8f10>
├── 'a' --> tensor([[[-0.3517,  1.2410,  0.3271],
│                    [ 1.8247, -0.2131, -0.2898]],
│           
│                   [[-0.3517,  1.2410,  0.3271],
│                    [ 1.8247, -0.2131, -0.2898]],
│           
│                   [[-0.3517,  1.2410,  0.3271],
│                    [ 1.8247, -0.2131, -0.2898]],
│           
│                   [[-0.3517,  1.2410,  0.3271],
│                    [ 1.8247, -0.2131, -0.2898]],
│           
│                   [[-0.3517,  1.2410,  0.3271],
│                    [ 1.8247, -0.2131, -0.2898]],
│           
│                   [[-0.3517,  1.2410,  0.3271],
│                    [ 1.8247, -0.2131, -0.2898]],
│           
│                   [[-0.3517,  1.2410,  0.3271],
│                    [ 1.8247, -0.2131, -0.2898]],
│           
│                   [[-0.3517,  1.2410,  0.3271],
│                    [ 1.8247, -0.2131, -0.2898]]])
└── 'x' --> <FastTreeValue 0x7f680beb8e50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.6 µs ± 84.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f680beb81c0>
├── 'a' --> tensor([[-0.3517,  1.2410,  0.3271],
│                   [ 1.8247, -0.2131, -0.2898],
│                   [-0.3517,  1.2410,  0.3271],
│                   [ 1.8247, -0.2131, -0.2898],
│                   [-0.3517,  1.2410,  0.3271],
│                   [ 1.8247, -0.2131, -0.2898],
│                   [-0.3517,  1.2410,  0.3271],
│                   [ 1.8247, -0.2131, -0.2898],
│                   [-0.3517,  1.2410,  0.3271],
│                   [ 1.8247, -0.2131, -0.2898],
│                   [-0.3517,  1.2410,  0.3271],
│                   [ 1.8247, -0.2131, -0.2898],
│                   [-0.3517,  1.2410,  0.3271],
│                   [ 1.8247, -0.2131, -0.2898],
│                   [-0.3517,  1.2410,  0.3271],
│                   [ 1.8247, -0.2131, -0.2898]])
└── 'x' --> <FastTreeValue 0x7f6787bccb80>
    └── 'c' --> tensor([[-0.4572, -1.7825, -2.1356, -0.1295],
                        [-0.2411,  0.5536,  0.3949, -0.7374],
                 

In [23]:
%timeit t_cat(trees)

29.3 µs ± 57.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.4 µs ± 112 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.4572, -1.7825, -2.1356, -0.1295],
                       [-0.2411,  0.5536,  0.3949, -0.7374],
                       [-0.1403, -0.3730,  0.8133, -1.4093]],
              
                      [[-0.4572, -1.7825, -2.1356, -0.1295],
                       [-0.2411,  0.5536,  0.3949, -0.7374],
                       [-0.1403, -0.3730,  0.8133, -1.4093]],
              
                      [[-0.4572, -1.7825, -2.1356, -0.1295],
                       [-0.2411,  0.5536,  0.3949, -0.7374],
                       [-0.1403, -0.3730,  0.8133, -1.4093]],
              
                      [[-0.4572, -1.7825, -2.1356, -0.1295],
                       [-0.2411,  0.5536,  0.3949, -0.7374],
                       [-0.1403, -0.3730,  0.8133, -1.4093]],
              
                      [[-0.4572, -1.7825, -2.1356, -0.1295],
                       [-0.2411,  0.5536,  0.3949, -0.7374],
                       [-0.1403, -0.3730,  0.8133, -1.4093]],

In [26]:
%timeit Batch.stack(batches)

86.4 µs ± 287 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.4572, -1.7825, -2.1356, -0.1295],
                      [-0.2411,  0.5536,  0.3949, -0.7374],
                      [-0.1403, -0.3730,  0.8133, -1.4093],
                      [-0.4572, -1.7825, -2.1356, -0.1295],
                      [-0.2411,  0.5536,  0.3949, -0.7374],
                      [-0.1403, -0.3730,  0.8133, -1.4093],
                      [-0.4572, -1.7825, -2.1356, -0.1295],
                      [-0.2411,  0.5536,  0.3949, -0.7374],
                      [-0.1403, -0.3730,  0.8133, -1.4093],
                      [-0.4572, -1.7825, -2.1356, -0.1295],
                      [-0.2411,  0.5536,  0.3949, -0.7374],
                      [-0.1403, -0.3730,  0.8133, -1.4093],
                      [-0.4572, -1.7825, -2.1356, -0.1295],
                      [-0.2411,  0.5536,  0.3949, -0.7374],
                      [-0.1403, -0.3730,  0.8133, -1.4093],
                      [-0.4572, -1.7825, -2.1356, -0.1295],
                   

In [28]:
%timeit Batch.cat(batches)

158 µs ± 355 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

303 µs ± 3.36 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
